In [1]:
import re
from collections import Counter
import numpy as np
import pandas as pd

In [2]:
def process_data(file_name):
    """
    Input: 
        A file_name which is found in your current directory. You just have to read it in. 
    Output: 
        words: a list containing all the words in the corpus (text file you read) in lower case. 
    """
    words = []
    with open(file_name) as file:
        data = file.read()
        lowerCase = data.lower()
        words = re.findall('\w+', lowerCase)
    return words

In [3]:
word_l = process_data('shakespeare.txt')
vocab = set(word_l)
print(f"There are {len(vocab)} unique words in the vocabulary.")

There are 6116 unique words in the vocabulary.


In [5]:
word_2 = process_data('big.txt')
vocab2 = set(word_2)
print(f"There are {len(vocab2)} unique words in the second vocabulary.")

There are 32198 unique words in the second vocabulary.


In [6]:
def get_count(word_l):
    '''
    Input:
        word_l: a set of words representing the corpus. 
    Output:
        word_count_dict: The wordcount dictionary where key is the word and value is its frequency.
    '''
    word_count_dict = {}  
    for word in word_l:
        word_count_dict[word] = word_count_dict.get(word,0)+1
        
    return word_count_dict

In [7]:
word_count_dict = get_count(word_l)
print(f"There are {len(word_count_dict)} key values pairs in first dictionary")
word_count_dict2 = get_count(word_2)
print(f"There are {len(word_count_dict2)} key values pairs in second dictionary")


There are 6116 key values pairs in first dictionary
There are 32198 key values pairs in second dictionary


In [9]:
def get_probs(word_count_dict):
    '''
    Input:
        word_count_dict: The wordcount dictionary where key is the word and value is its frequency.
    Output:
        probs: A dictionary where keys are the words and the values are the probability that a word will occur. 
    '''
    probs = {}
    m = sum(word_count_dict.values())
    for word,count in word_count_dict.items():
        probs[word] = count/m
    return probs

In [10]:
probs = get_probs(word_count_dict)
probs2 = get_probs(word_count_dict2)

In [11]:
def delete_letter(word, verbose=False):
    '''
    Input:
        word: the string/word for which you will generate all possible words 
                in the vocabulary which have 1 missing character
    Output:
        delete_l: a list of all possible strings obtained by deleting 1 character from word
    '''
    delete_l = []
    split_l = []
    
    split_l = [(word[:i],word[i:]) for i in range(len(word)+1)]
    delete_l = [L+R[1:] for L,R in split_l if R]
    
    if verbose: print(f"input word {word}, \nsplit_l = {split_l}, \ndelete_l = {delete_l}")
    return delete_l

In [12]:
def switch_letter(word, verbose=False):
    '''
    Input:
        word: input string
     Output:
        switches: a list of all possible strings with one adjacent charater switched
    ''' 
    
    switch_l = []
    split_l = []

    split_l = [(word[:i],word[i:]) for i in range(len(word)+1)]
    switch_l = [L[:-1]+R[0]+L[-1]+R[1:] for L,R in split_l if R and L]
    
    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nswitch_l = {switch_l}") 
    return switch_l

In [13]:
def replace_letter(word, verbose=False):
    '''
    Input:
        word: the input string/word 
    Output:
        replaces: a list of all possible strings where we replaced one letter from the original word. 
    ''' 
    
    letters = 'abcdefghijklmnopqrstuvwxyz'
    replace_set = []
    split_l = []
    
    split_l = [(word[:i],word[i:]) for i in range(len(word)+1)]
    replace_set = [ L[:-1] + c + R for L,R in split_l if L for c in letters if c != L[-1]]
    replace_l = sorted(list(replace_set))
    
    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nreplace_l {replace_l}")   
    return replace_l

In [14]:
def insert_letter(word, verbose=False):
    '''
    Input:
        word: the input string/word 
    Output:
        inserts: a set of all possible strings with one new letter inserted at every offset
    ''' 
    letters = 'abcdefghijklmnopqrstuvwxyz'
    insert_l = []
    split_l = []

    split_l = [(word[:i],word[i:]) for i in range(len(word)+1)]
    insert_l = [ L + c + R for L,R in split_l for c in letters]

    if verbose: print(f"Input word {word} \nsplit_l = {split_l} \ninsert_l = {insert_l}")
    return insert_l

In [15]:
def edit_one_letter(word, allow_switches = True):
    """
    Input:
        word: the string/word for which we will generate all possible wordsthat are one edit away.
    Output:
        edit_one_set: a set of words with one possible edit. Please return a set. and not a list.
    """
    
    edit_one_set = set()
    f1 = []
    f2 = []
    f3 = []
    f4 = []

    f1 = insert_letter(word, verbose=False)
    f2 = replace_letter(word, verbose=False)
    f3 = delete_letter(word, verbose=False)
    if allow_switches:
        f4 = switch_letter(word, verbose=False)
    edit_one_set = set(f1+f2+f3+f4)    
    
    return edit_one_set

In [16]:
def edit_two_letters(word, allow_switches = True):
    '''
    Input:
        word: the input string/word 
    Output:
        edit_two_set: a set of strings with all possible two edits
    '''
    
    edit_two_set = set()
    
    oneL = edit_one_letter(word, allow_switches = True)
    for i in oneL: 
        edit_two_set = set(edit_one_letter(i, allow_switches = True)) | edit_two_set
    
    return edit_two_set

In [17]:
def get_corrections(word, probs, vocab, n=2, verbose = False):
    '''
    Input: 
        word: a user entered string to check for suggestions
        probs: a dictionary that maps each word to its probability in the corpus
        vocab: a set containing all the vocabulary
        n: number of possible word corrections you want returned in the dictionary
    Output: 
        n_best: a list of tuples with the most probable n corrected words and their probabilities.
    '''
    
    suggestions = []
    n_best = []
    
    set_1 = edit_one_letter(word, allow_switches = True)
    set_2 = edit_two_letters(word, allow_switches = True)
    if word in vocab:
        suggestions.append((word,probs[word]))
    else:
        suggestions = [(i,probs[i]) for i in set_1 if i in vocab] or [(i,probs[i]) for i in set_2 if i in vocab] or [(word,0)]
    n_best = sorted(suggestions, key = lambda x : x[-1], reverse = True)[:n]            
    
    if verbose: print("entered word = ", word, "\nsuggestions = ", suggestions)
    return n_best

In [22]:
# Test the implementation
my_word = 'helol' 
tmp_corrections = get_corrections(my_word, probs, vocab, 2, verbose=True) # keep verbose=True
for i, word_prob in enumerate(tmp_corrections):
    print(f"word {i}: {word_prob[0]}, probability {word_prob[1]:.6f}")


entered word =  helol 
suggestions =  [('hell', 0.0001678666020069385)]
word 0: hell, probability 0.000168


In [24]:
# Test the implementation
my_word = 'helo1' 
tmp_corrections = get_corrections(my_word, probs2, vocab2, 2, verbose=True) # keep verbose=True
for i, word_prob in enumerate(tmp_corrections):
    print(f"word {i}: {word_prob[0]}, probability {word_prob[1]:.6f}")


entered word =  helo1 
suggestions =  [('hero', 4.9301487560338295e-05), ('hell', 3.585562731660967e-06), ('below', 0.0001030849285352528), ('helps', 4.481953414576209e-06), ('hello', 8.963906829152417e-07), ('felon', 8.963906829152417e-07), ('helm', 8.963906829152417e-07), ('helen', 5.378344097491451e-06), ('help', 0.0002061698570705056), ('halo', 2.6891720487457255e-06), ('held', 0.0002572641259966744), ('melon', 7.171125463321934e-06)]
word 0: held, probability 0.000257
word 1: help, probability 0.000206
